In [3]:
import pandas as pd

# Import q_stats

# Model import

In [5]:
import pickle
my_pipeline = pickle.load(open('../models/xgboost_pipe_M1.pkl',"rb"))  ## wb= write bites : le b est important

In [12]:
pipeline_features_list=pd.read_csv('../models/xgboost_pipe_M1_features_list')

In [13]:
pipeline_features_list

,feature
0,user_personalized_qstat_knowing_had_explanatio...
1,user_avg_score_cum_on_this_part
2,user_correct_answers_cum_on_this_part
3,qstats_answered_correctly
4,user_avg_score_cum
5,qstats_prior_question_had_explanation
6,already_seen
7,user_activity_cumcount
8,at_least_one_lesson


# Initialisation utilisateur débutant

In [16]:
user_history_empty=pd.DataFrame({'content_type_id':[-1],
                             'content_id':[-1],
                             'user_activity_cumcount':[-1],
                             'user_avg_score_cum':[0.499],
                             'user_correct_answers_cum':[0],
                             'user_avg_score_cum_part1':[0.499],
                             'user_avg_score_cum_part2':[0.499],
                             'user_avg_score_cum_part3':[0.499],
                             'user_avg_score_cum_part4':[0.499],
                             'user_avg_score_cum_part5':[0.499],
                             'user_avg_score_cum_part6':[0.499],
                             'user_avg_score_cum_part7':[0.499],
                             'user_correct_answers_cum_part1':[0],
                             'user_correct_answers_cum_part2':[0],
                             'user_correct_answers_cum_part3':[0],
                             'user_correct_answers_cum_part4':[0],
                             'user_correct_answers_cum_part5':[0],
                             'user_correct_answers_cum_part6':[0],
                             'user_correct_answers_cum_part7':[0],
                             'at_least_one_lesson':[0],
                             'user_personalized_qstat_knowing_had_explanation_or_not':[-1],
                             'user_avg_score_cum_on_this_part':[0.499],
                             'user_correct_answers_cum_on_this_part':[0],
                             'already_seen':[-1],
                             'qstats_answered_correctly':[-1],
                             'qstats_prior_question_had_explanation':[-1],
                             'answered_correctly':[-1]
                          })
user_history

,content_type_id,content_id,user_activity_cumcount,user_avg_score_cum,user_correct_answers_cum,user_avg_score_cum_part1,user_avg_score_cum_part2,user_avg_score_cum_part3,user_avg_score_cum_part4,user_avg_score_cum_part5,...,user_correct_answers_cum_part6,user_correct_answers_cum_part7,at_least_one_lesson,user_personalized_qstat_knowing_had_explanation_or_not,user_avg_score_cum_on_this_part,user_correct_answers_cum_on_this_part,already_seen,qstats_answered_correctly,qstats_prior_question_had_explanation,answered_correctly
0,-1,-1,0,0.499,0,0.499,0.499,0.499,0.499,0.499,...,0,0,0,-1,0.499,0,-1,-1,-1,-1


# User history update fonction

In [ ]:
def user_history_update(content_type_id, content_id, data_qstats, user_history=None):
    '''Crée ou met à jour l'hisorique d'un utilisateur, stockée dans un df'''
    
    user_history_empty=pd.DataFrame({ # following columns depend of previous history of the user : 
                             ### TO BE UPDATED ###
                             'user_activity_cumcount':[-1],
                             'user_avg_score_cum':[0.499],
                             'user_correct_answers_cum':[0],
                             'user_avg_score_cum_part1':[0.499],
                             'user_avg_score_cum_part2':[0.499],
                             'user_avg_score_cum_part3':[0.499],
                             'user_avg_score_cum_part4':[0.499],
                             'user_avg_score_cum_part5':[0.499],
                             'user_avg_score_cum_part6':[0.499],
                             'user_avg_score_cum_part7':[0.499],
                             'user_correct_answers_cum_part1':[0],
                             'user_correct_answers_cum_part2':[0],
                             'user_correct_answers_cum_part3':[0],
                             'user_correct_answers_cum_part4':[0],
                             'user_correct_answers_cum_part5':[0],
                             'user_correct_answers_cum_part6':[0],
                             'user_correct_answers_cum_part7':[0],
                             'at_least_one_lesson':[0],
                             # following columns are pure question stats : 
                             ### TO BE IMPORTED ###
                             'content_type_id':[-1],
                             'content_id':[-1],
                             'part_q_l':[-1]
                             'qstats_answered_correctly':[-1],
                             'qstats_prior_question_had_explanation':[-1],
                             'qstats_answered_correctly_knowing_having_had_explanation':[-1],
                             'qstats_answered_correctly_knowing_having_not_had_explanation':[-1],
                             # following columns depend of the current question AND the hisory of user
                             ### TO BE COMPUTED ###
                             'user_personalized_qstat_knowing_had_explanation_or_not':[-1],
                             'already_seen':[-1],
                             'user_avg_score_cum_on_this_part':[0.499],
                             'user_correct_answers_cum_on_this_part':[0],
                             # the following line is the prediction to be made
                             ### TO BE PREDICTED ###
                             'answered_correctly':[-1]
                          })
    
    if not type(user_history)==pd.DataFrame:
        user_history=user_history_empty
    
    last_line=user_history.iloc[-1]
    new_line =last_line.copy()
    new_line['user_activity_cumcount'] = last_line['user_activity_cumcount'] + 1
    
    last_content_type_id=user_history.iloc[-1]['content_type_id']
    if not last_content_type_id==1:# TODO : coder le comportement de cette fonction quand elle arrive sur une lecture

            new_line['at_least_one_lesson']=1
    
    elif last_content_type_id==0:
        
        ### USER HISTORY FEATURES ###
            
            new_line['user_correct_answers_cum'] = last_line['user_correct_answers_cum']\
                                                 + last_line['answered_correctly']
            new_user_questions_count             = last_line['user_correct_answers_cum']\
                                                 / last_line['user_avg_score_cum']\
                                                 + 1
            new_line['user_avg_score_cum']       = new_line['user_correct_answers_cum']\
                                                 / new_user_questions_count
            
            for part in range(1,8):
                new_line[f'user_correct_answers_cum_part{part}'] = last_line[f'user_correct_answers_cum_part{part}']\
                                                                 + last_line[f'answered_correctly_part{part}']
                vars()[f'new_user_questions_count_part{part}']   = last_line[f'user_correct_answers_cum_part{part}']\
                                                                 / last_line[f'user_avg_score_cum_part{part}']\
                                                                 + 1
                new_line[f'user_avg_score_cum_part{part}']       = new_line[f'user_correct_answers_cum_part{part}']\
                                                                 / vars()[f'new_user_questions_count_part{part}']                
        
        ### CURRENT QUESTION IMPORT  
                                         

        new_line= pd.DataFrame({'user_id':user,
                             'answered_correctly':row['answered_correctly'],
                             'correctly_answered_cumcount':new_correctly_answered_cumcount,
                             'uncorrectly_answered_cumcount':new_uncorrectly_answered_cumcount,
                             'user_avg_score_cum':new_user_avg_score_cum,
                             'qstats_answered_correctly':data_qstats.loc[content].qstats_answered_correctly,
                                           },
                              index=[-1])
        user_history=pd.concat([user_history,new_line],axis=0,ignore_index=True)

    return user_history

# Loop

In [15]:
loop_length=1000
question_selection_strategy='random'

In [ ]:
for i in range(loop_length):
    

In [17]:
range(1,7)

range(1, 7)

In [19]:
for i in range(1,7):
    print(i)

1
2
3
4
5
6


In [21]:
i=9
vars()[f'machin{i}']=99
machin9

99